In [26]:
%reload_kedro
import pandas as pd
pd.set_option('display.max_columns', None)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from joblib import Parallel, delayed

from tqdm import tqdm

[06/12/24 13:06:03] INFO     Resolved project path as:                                              ]8;id=418634;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=165465;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#164\164]8;;\
                             C:\Users\yair0\OneDrive\Escritorio\Projectbreitong.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/12/24 13:06:04] INFO     Kedro project breitong                                                 ]8;id=391640;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=619849;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=630174;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=515035;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=600825;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=30233;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

In [27]:
import pandas as pd

def remove_high_null_columns(df: pd.DataFrame, threshold: float = 0.5) -> pd.DataFrame:
    """
    Remove columns from a DataFrame that have more than a specified threshold of null values.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    threshold (float): The threshold for the percentage of null values. Columns with a percentage of null values greater than this threshold will be removed. Default is 0.5 (50%).

    Returns:
    pd.DataFrame: The DataFrame with columns removed that had more than the specified threshold of null values.
    """
    # Calculate the percentage of null values for each column
    null_percentage = df.isnull().mean()
    # Filter out columns that have more than the specified threshold of null values
    columns_to_keep = null_percentage[null_percentage <= threshold].index
    # Return the DataFrame with the selected columns
    return df[columns_to_keep]


In [28]:
df_champs = catalog.load("champions")
df_grades_2023 = catalog.load("grades_2023")
df_questionnaires = catalog.load("questionnaire")
df_leadership = catalog.load("leadership")
df_primary_psycho = catalog.load("primary_psycho")
df_secondary_psycho = catalog.load("secondary_psycho")


[06/12/24 13:06:15] INFO     Loading data from champions (ExcelDataset)...                      ]8;id=680957;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=734661;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

                    INFO     Loading data from grades_2023 (ExcelDataset)...                    ]8;id=104503;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=471777;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

[06/12/24 13:06:16] INFO     Loading data from questionnaire (ExcelDataset)...                  ]8;id=57538;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=129904;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

[06/12/24 13:06:17] INFO     Loading data from leadership (ExcelDataset)...                     ]8;id=151604;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=383181;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

                    INFO     Loading data from primary_psycho (ExcelDataset)...                 ]8;id=928307;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=795189;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

[06/12/24 13:06:18] INFO     Loading data from secondary_psycho (ExcelDataset)...               ]8;id=782801;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=911765;file://C:\Users\yair0\anaconda3\envs\breitong\lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

### Parse Cuestionario

In [20]:
def get_sentiment_score(text, analyzer):
    try:
        return analyzer.polarity_scores(text)['compound']
    except Exception as e:
        return 0.0

def get_sentiment(col: pd.Series, analyzer: SentimentIntensityAnalyzer, n_jobs=-1):
    """
    Get sentiment score for a given text series using parallel processing
    """
    sentiment_scores = Parallel(n_jobs=n_jobs)(delayed(get_sentiment_score)(text, analyzer) for text in col)
    return pd.Series(sentiment_scores, index=col.index)

In [ ]:
df_questionnaires_final = remove_high_null_columns(df_questionnaires)
analyzer = SentimentIntensityAnalyzer()
cols_to_sentiment = [
    'Champ para talleres virtuales se puede conectar de',
    '¿Qué tema te interesa Champ?',
    '¿Con quienes vives?  \nPor ejemplo: Con mi padres y hermanos',
    '¿Cuáles son tus pasatiempos favoritos?  Por ejemplo: Leer y jugar videojuegos',
    '¿Cuáles son tus cursos favoritos? \nPor ejemplo: Matemática y Comunicación',
    '¿Cuál es tu deporte favorito? \nPor ejemplo: Fútbol',
    '¿Qué te gustaría estudiar o ser de grande? \nPor ejemplo: Doctor',
    '¿Qué te gustaría hacer para mejorar el Perú?',
    '¿Te gustaría estudiar en una universidad extranjera?',
    'Cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023',
    'En Perú Champs nos preocupa tu bienestar. ¿Hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? Este espacio es confidencial y no afecta tu beca',
    '¿Tiene otro champ?', 'DNI DEL APODERADO ENCRIPTADO',
    'Qué relación tiene con Champ',
    'Su Champ tiene un apoderada/o adicional a Usted',
    '¿Tienes un negocio propio o emprendimiento?',
    'La conexión a Internet en su casa es', 'Tu internet es',
    '¿Algún familiar con el que viva tiene alguna condición, discapacidad o enfermedad crónica que requiera de cuidados constante?',
    'Cuéntanos brevemente cómo se encuentran actualmente',
    '¿Quién apoya a su Champ con sus tareas?'
    ]

for col in cols_to_sentiment:
    df_questionnaires_final[col + "_sentiment"] = get_sentiment(df_questionnaires_final[col].astype(str), analyzer)
    print(col + " done")

df_questionnaires_final


In [31]:
df_questionnaires_final.to_csv("data/01_raw/df_questionnaires_sense.csv", index=False)

In [33]:
df_leadership_final = remove_high_null_columns(df_leadership)
analyzer = SentimentIntensityAnalyzer()
cols_to_sentiment = [
    '¿Qué tema te interesa Champ?',
    '¿Con quienes vives?  \nPor ejemplo: Con mi padres y hermanos',
    '¿Cuáles son tus pasatiempos favoritos?  Por ejemplo: Leer y jugar videojuegos',
    '¿Cuáles son tus cursos favoritos? \nPor ejemplo: Matemática y Comunicación',
    '¿Cuál es tu deporte favorito? \nPor ejemplo: Fútbol',
    '¿Qué te gustaría estudiar o ser de grande? \nPor ejemplo: Doctor',
    '¿Qué te gustaría hacer para mejorar el Perú?',
    '¿Te gustaría estudiar en una universidad extranjera?',
    'Cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023',
    'En Perú Champs nos preocupa tu bienestar. ¿Hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? Este espacio es confidencial y no afecta tu beca'
    ]

for col in cols_to_sentiment:
    df_leadership_final[col + "_sentiment"] = get_sentiment(df_leadership_final[col].astype(str), analyzer)
    print(col + " done")


df_leadership_final.to_csv("data/01_raw/df_leadership_sense.csv", index=False)
df_leadership_final

¿Qué tema te interesa Champ? done
¿Con quienes vives?  
Por ejemplo: Con mi padres y hermanos done
¿Cuáles son tus pasatiempos favoritos?  Por ejemplo: Leer y jugar videojuegos done
¿Cuáles son tus cursos favoritos? 
Por ejemplo: Matemática y Comunicación done
¿Cuál es tu deporte favorito? 
Por ejemplo: Fútbol done
¿Qué te gustaría estudiar o ser de grande? 
Por ejemplo: Doctor done
¿Qué te gustaría hacer para mejorar el Perú? done
¿Te gustaría estudiar en una universidad extranjera? done
Cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023 done
En Perú Champs nos preocupa tu bienestar. ¿Hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? Este espacio es confidencial y no afecta tu beca done


,DNI ENCRIPTADO,ESTADO,NRE (NIVEL DE RIESGO EMOCIONAL),CANTERA,AÑO DE INGRESO,REGIÓN,SEDE,SEXO,GRADO,FECHA NACIMIENTO,EDAD,CONDICIÓN DE SALUD,¿Qué tema te interesa Champ?,¿Con quienes vives? \nPor ejemplo: Con mi padres y hermanos,¿Cuáles son tus pasatiempos favoritos? Por ejemplo: Leer y jugar videojuegos,¿Cuáles son tus cursos favoritos? \nPor ejemplo: Matemática y Comunicación,¿Cuál es tu deporte favorito? \nPor ejemplo: Fútbol,¿Qué te gustaría estudiar o ser de grande? \nPor ejemplo: Doctor,¿Qué te gustaría hacer para mejorar el Perú?,¿Te gustaría estudiar en una universidad extranjera?,"Cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023","En Perú Champs nos preocupa tu bienestar. ¿Hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? Este espacio es confidencial y no afecta tu beca",¿Qué tema te interesa Champ?_sentiment,¿Con quienes vives? \nPor ejemplo: Con mi padres y hermanos_sentiment,¿Cuáles son tus pasatiempos favoritos? Por ejemplo: Leer y jugar videojuegos_sentiment,¿Cuáles son tus cursos favoritos? \nPor ejemplo: Matemática y Comunicación_sentiment,¿Cuál es tu deporte favorito? \nPor ejemplo: Fútbol_sentiment,¿Qué te gustaría estudiar o ser de grande? \nPor ejemplo: Doctor_sentiment,¿Qué te gustaría hacer para mejorar el Perú?_sentiment,¿Te gustaría estudiar en una universidad extranjera?_sentiment,"Cuéntanos si ganaste un reconocimiento, concurso, beca, tienes una organización o has sido voluntario/a en el 2023_sentiment","En Perú Champs nos preocupa tu bienestar. ¿Hay algo que quieras compartirnos sobre algo que fue difícil en el 2023, algo que te hiciera sentir triste o frustrado/a? Este espacio es confidencial y no afecta tu beca_sentiment"
0,NZDDSEZR,Matriculado,SIN EV.,Deportiva,2024,Lima,Bertello,M,Undécimo,2008-01-16,16,Ninguno,Acción por el clima,CON MI PADRE Y MIS HERMANAS,JUGAR TENIS,MATEMATICA,VELA- DEPORTE NAUTICO,ECONOMIA,SER UN BUEN PROFESIONAL EN ECONOMIA,Sí,CAMPEON SUDAMERICANO DE OPTIMIST -VELA,EL COLEGIO DONDE ESTUVE,0.0000,0.0,0.34,0.0,0.0,0.0,0.4404,0.296,0.8679,0.0000
1,NARYEWZA,Matriculado,BAJO,Académica,2024,Lima,Surco Ambrosio,M,Sexto,2012-06-18,11,Ninguno,Acción por el clima,"Padres, hermano y hermana",Leer y jugar ajedrez,Matemática,Fútbol,Ingeniero,"Que no haya discriminación, que todos se respe...",Sí,"Gane premio de la feria de ciencias dos años, ...",El cambiar de colegio. Pero se que el cambio e...,0.0000,0.0,0.34,0.0,0.0,0.0,-0.0516,0.296,0.8519,0.8271
2,RRNASAER,Matriculado,BAJO,Artística,2024,Lima,Ate Puruchuco,M,Sexto,2012-05-01,11,"Asperger, A veces le sangra la nariz, TDH",Acción por el clima,Con mamá y hermanos,Jugar con mis legos,Matemática y comunicación,Fútbol,Ingeniero ambiental,Formar grupos de ayuda comunitaria para ayudar...,Sí,"Me dieron un reconocimiento por una poesía, lo...",Mi papá no confía en que pueda lograr salir ad...,0.0000,0.0,0.34,0.0,0.0,0.0,0.6597,0.296,0.6115,-0.8690
3,RRZZYSII,Matriculado,SIN EV.,Empresa (Académica),2024,Lima,SJL Arabiscos,F,Octavo,2010-06-16,13,Ninguno,Salud y bienestar,PADRES Y HERMANO,LEER Y JUGAR,"INGLES, MATEMATICA",BASQUET,ZOOTECNIA O MEDICINA,DISMINUIR LA DELINCUENCIA Y MAS TRABAJO PARA T...,Sí,NO,NO,0.4404,0.0,0.34,0.0,0.0,0.0,-0.5423,0.296,0.0000,0.0000
4,NAZRYRZN,Matriculado,MEDIO,Académica,2024,Junin,Huancayo,M,Séptimo,2011-12-09,12,Rinitis alérgica,Acción por el clima,Con mi mamá y hermana,Leer,Comunicación e historia,Natación,Abogada,Plantar árboles para mejorar el medio ambiente,Sí,Ninguna,La muerte de mi mascota,0.0000,0.0,0.00,0.0,0.0,0.0,0.4404,0.296,0.0000,-0.5994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,NAZYWDNE,Matriculado,SIN EV.,Académica,2022,Arequipa,Cerro Colorado,F,Séptimo,2011-12-29,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0,0.00,0.0,0.0,0.0,0.0000,0.000,0.0000,0.0000
